In [8]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "context_length": 256,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    "qkv_bias": False
}

In [9]:
import torch
import torch.nn as nn
import tensorflow as tf

class DummyGPTModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.tok_emb = nn.Embedding(config['vocab_size'], config['emb_dim'])
        self.pos_emb = nn.Embedding(config['context_length'], config['emb_dim'])
        self.drop_emb = nn.Dropout(config['drop_rate'])
        
        self.transformer_blocks = nn.Sequential(
            *[DummyTransformerBlock(config) for _ in range(config['n_layers'])]
        )
        self.final_norm = DummyLayerNorm(config['emb_dim'])
        self.out_head = nn.Linear(config['emb_dim'], config['vocab_size'], bias=False)
        
    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device)) # organiza em posicoes 0, 1, 2, 3 etc
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.transformer_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)   # (num_tokens, vocab_size)
        return logits

class DummyTransformerBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
    
    def forward(self, x):
        return x

class DummyLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()
    
    def forward(self, x):
        return x

In [10]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "Every effort moves you"
txt2 = "Every day holds a"
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [11]:
torch.manual_seed(123)
model = DummyGPTModel(GPT_CONFIG_124M)
logits = model(batch)
print("Output shape: ", logits.shape)
print(logits)

Output shape:  torch.Size([2, 4, 50257])
tensor([[[ 0.6442,  0.4002, -0.6301,  ..., -1.7908,  1.7656,  0.7682],
         [ 0.4882, -1.3109, -1.0632,  ..., -0.3465,  0.5326, -0.2410],
         [-0.2399,  1.1010,  0.5612,  ..., -0.1098, -0.0362, -1.2148],
         [-1.0398, -0.0377,  1.6094,  ..., -1.7665, -1.2933, -1.6788]],

        [[ 0.9140,  0.5402, -0.9668,  ..., -1.1724,  1.3664,  0.5812],
         [-0.5798, -0.3736, -1.2043,  ..., -1.7816,  0.4513,  0.0474],
         [ 0.1278, -0.4755, -0.8627,  ..., -0.6784,  0.1200, -0.3627],
         [-1.0198,  1.5377,  1.0421,  ..., -0.9028, -0.9430, -0.5627]]],
       grad_fn=<UnsafeViewBackward0>)


In [12]:
torch.manual_seed(123)
batch_example = torch.randn(2, 5)
layer = nn.Sequential(nn.Linear(5, 6), nn.ReLU())
out = layer(batch_example)
print(out)

tensor([[0.2260, 0.3470, 0.0000, 0.2216, 0.0000, 0.0000],
        [0.2133, 0.2394, 0.0000, 0.5198, 0.3297, 0.0000]],
       grad_fn=<ReluBackward0>)


In [13]:
# Se nao colocasse keepdim, o tensor de média seria um vetor de 2 dimensões ([x1, x2]) ao invés de uma matriz 2x1
mean = out.mean(dim=-1, keepdim=True) # dim = -1 para pegar a média por coluna
var = out.var(dim=-1, keepdim=True)
print("Mean: \n", mean)
print("Variance: \n", var)

Mean: 
 tensor([[0.1324],
        [0.2170]], grad_fn=<MeanBackward1>)
Variance: 
 tensor([[0.0231],
        [0.0398]], grad_fn=<VarBackward0>)


In [14]:
out_norm = (out - mean) / torch.sqrt(var) # media zero e variancia 1 para evitar gradient vanishing / exploding
mean = out_norm.mean(dim=-1, keepdim=True)
var = out_norm.var(dim=-1, keepdim=True)
print("Normalized layer outputs: \n", out_norm)
print("Mean: \n", mean)
print("Variance: \n", var)

Normalized layer outputs: 
 tensor([[ 0.6159,  1.4126, -0.8719,  0.5872, -0.8719, -0.8719],
        [-0.0189,  0.1121, -1.0876,  1.5173,  0.5647, -1.0876]],
       grad_fn=<DivBackward0>)
Mean: 
 tensor([[9.9341e-09],
        [1.9868e-08]], grad_fn=<MeanBackward1>)
Variance: 
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)


In [15]:
torch.set_printoptions(sci_mode=False)
print("Mean: \n", mean)
print("Variance: \n", var)

Mean: 
 tensor([[    0.0000],
        [    0.0000]], grad_fn=<MeanBackward1>)
Variance: 
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)


In [16]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5 # pequena constante para prevenir divisão por zero durante normalização
        
        # scale e shift são parametros treinaveis c mesma dimensao que o input
        # que o llm ajusta automaticamente durante o treino se melhorar performance
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))
    
    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

In [17]:
ln = LayerNorm(emb_dim=5)
out_ln = ln(batch_example)
mean = out_ln.mean(dim=-1, keepdim=True)
var = out_ln.var(dim=-1, keepdim=True, unbiased=False)
print("Mean: \n", mean)
print("Variance: \n", var)

Mean: 
 tensor([[    -0.0000],
        [     0.0000]], grad_fn=<MeanBackward1>)
Variance: 
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)


In [18]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))

In [19]:
class FeedForward(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.layers = nn.Sequential(
            # Dimensões da camada linear são: dimensão do input, dimensão do output
            nn.Linear(config['emb_dim'], 4 * config['emb_dim']), # 768 dim de input e 4 * 768 de output
            GELU(), # ativação
            nn.Linear(4 * config['emb_dim'], config['emb_dim']),
        )
    
    def forward(self, x):
        return self.layers(x)

In [20]:
ffn = FeedForward(GPT_CONFIG_124M)
x = torch.rand(2, 3, 768) # input com dois batches
out = ffn(x)
print(out.shape)

torch.Size([2, 3, 768])


In [21]:
class ExampleDeepNeuralNetwork(nn.Module):
    def __init__(self, layer_sizes, use_shortcut):
        super().__init__()
        self.use_shortcut = use_shortcut
        self.layers = nn.ModuleList([
            # nn.Linear tem (dim input, dim output), dim input e output sao o numero de neuronios
            nn.Sequential(nn.Linear(layer_sizes[0], layer_sizes[1]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[1], layer_sizes[2]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[2], layer_sizes[3]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[3], layer_sizes[4]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[4], layer_sizes[5]), GELU())
        ])
    
    def forward(self, x):

        for layer in self.layers:
            layer_output = layer(x)
            
            if self.use_shortcut and x.shape == layer_output.shape:
                x = x + layer_output
            else:
                x = layer_output
        return x

In [22]:
layer_sizes = [3, 3, 3, 3, 3, 1] # 5 camadas com 3 neuronios e a final tem 1
sample_input = torch.tensor([[1., 0., -1.]])
torch.manual_seed(123)
model_without_shortcut = ExampleDeepNeuralNetwork(layer_sizes, use_shortcut=False)

In [23]:
def print_gradients(model, x):
    output = model(x)
    target = torch.tensor([[0.]])
    
    loss = nn.MSELoss()
    loss = loss(output, target)
    
    # backward pass para calcular gradientes
    loss.backward()
    
    for name, param in model.named_parameters():
        if 'weight' in name:
            print(f"{name} tem media de gradiente {param.grad.abs().mean().item()}")

In [24]:
print_gradients(model_without_shortcut, sample_input) # demonstra problema do vanishing gradient

layers.0.0.weight tem media de gradiente 0.00020173587836325169
layers.1.0.weight tem media de gradiente 0.00012011159560643137
layers.2.0.weight tem media de gradiente 0.0007152040489017963
layers.3.0.weight tem media de gradiente 0.0013988736318424344
layers.4.0.weight tem media de gradiente 0.005049645435065031


In [25]:
torch.manual_seed(123)
model_with_shortcut = ExampleDeepNeuralNetwork(layer_sizes, use_shortcut=True)
print_gradients(model_with_shortcut, sample_input)

layers.0.0.weight tem media de gradiente 0.22169798612594604
layers.1.0.weight tem media de gradiente 0.20694111287593842
layers.2.0.weight tem media de gradiente 0.3289700150489807
layers.3.0.weight tem media de gradiente 0.26657330989837646
layers.4.0.weight tem media de gradiente 1.3258544206619263


In [26]:
class MultiHeadAttention(torch.nn.Module):
    
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out deve ser divisível por num_heads"
    
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        
        self.W_query = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = torch.nn.Linear(d_out, d_out) # camada linear para combinar outputs de cada head
        self.dropout = torch.nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1),
        )
    
    def forward(self, x):
        b, num_tokens, d_in = x.shape
        
        keys = self.W_key(x)
        values = self.W_value(x)
        queries = self.W_query(x)
        
        # dividindo matrizes implicitamente ao adicionar dimensão num_heads
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
        
        # transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        # agrupando por head, cada uma vai ter uma matriz de attention score
        # inicialmente agrupava por número de tokens, mas não é eficiente para calcularmos attention scores para cada cabeça paralelamente
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)
        
        # computando attention scores
        # após a multiplicação, fica com dimensões (b, num_heads, num_tokens, num_tokens), e simboliza o valor de atenção que precisa dar p cada palavra
        attn_scores = queries @ keys.transpose(2, 3) # o que é importante é num_tokens e head_dim para calcularmos produto escalar
        
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        
        # attention weights tem dimensões (b, num_heads, num_tokens, num_tokens)
        # values tem dimensões (b, num_heads, num_tokens, head_dim)
        # Attention weights e values tem a mesma dimensão como linhas, então é possível multiplicar
        # dimensão final: (b, num_heads, num_tokens, head_dim)
        # com isso, transpomos a matriz para termos (b, num_tokens, num_heads, head_dim) para podermos dar merge no num_heads e head_dim
        context_vector = (attn_weights @ values).transpose(1, 2)
        
        # combinando heads, self_d_out = self.num_heads * self_head_dim
        # Fica com dimensão (b, num_tokens, d_out)
        # contiguous() serve para colocar as matrizes no mesmo bloco de memória para fazermos o merge
        context_vector = context_vector.contiguous().view(b, num_tokens, self.d_out)
        context_vector = self.out_proj(context_vector) # projeção opcional
        
        return context_vector

In [27]:
class TransformerBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in = config['emb_dim'],
            d_out = config['emb_dim'],
            context_length = config['context_length'],
            num_heads = config['n_heads'],
            dropout = config['drop_rate'],
            qkv_bias = config['qkv_bias']
        )
        self.ff = FeedForward(config)
        self.norm1 = LayerNorm(config['emb_dim'])
        self.norm2 = LayerNorm(config['emb_dim'])
        self.drop_shortcut = nn.Dropout(config['drop_rate'])
        
    def forward(self, x):
        # shortcut connection para attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        x = self.drop_shortcut(x)
        x = x + shortcut
        
        # shortcut connection para feed forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut
        
        return x

In [28]:
torch.manual_seed(123)
x = torch.rand(2, 4, 768)
block = TransformerBlock(GPT_CONFIG_124M)
output = block(x)
print("Input shape: ", x.shape)
print("Output shape: ", output.shape)

Input shape:  torch.Size([2, 4, 768])
Output shape:  torch.Size([2, 4, 768])


In [73]:
import torch
import torch.nn as nn

class GPTModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.tok_emb = nn.Embedding(config['vocab_size'], config['emb_dim'])
        self.pos_emb = nn.Embedding(config['context_length'], config['emb_dim'])
        self.drop_emb = nn.Dropout(config['drop_rate'])
        
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(config) for _ in range(config['n_layers'])]
        )
        self.final_norm = LayerNorm(config['emb_dim'])
        self.out_head = nn.Linear(config['emb_dim'], config['vocab_size'], bias=False)
        
    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device)) # organiza em posicoes 0, 1, 2, 3 etc
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)   # (num_tokens, vocab_size)
        return logits

In [30]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
out = model(batch)
print("Input batch:\n", batch)
print("Output shape: ", out.shape)
print(out)

Input batch:
 tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])
Output shape:  torch.Size([2, 4, 50257])
tensor([[[-0.4708,  0.5737, -0.5967,  ...,  0.2019, -0.5665,  0.1800],
         [-0.3895, -0.1978, -0.8885,  ...,  0.2242, -1.2341,  0.1752],
         [ 0.6973, -0.3432, -0.6080,  ...,  0.3747, -0.6967,  0.1088],
         [-0.2962, -0.6957, -1.1371,  ...,  0.3579,  0.3058, -0.2915]],

        [[-0.1514,  0.3329, -0.9740,  ..., -0.1368, -0.6974, -0.1851],
         [-0.4894, -0.3492, -0.9759,  ...,  0.2951, -0.3396,  0.2109],
         [ 0.5082, -0.1425,  0.2549,  ...,  0.1618,  0.1304, -0.3092],
         [-0.4146, -0.0514, -0.5187,  ..., -0.1869, -0.1303, -0.4969]]],
       grad_fn=<UnsafeViewBackward0>)


In [31]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Número total de parâmetros: {total_params:,}") # tem mais parâmetros pois utiliza token embeddings na camada de output

Número total de parâmetros: 162,419,712


In [32]:
total_params_gpt2 = total_params - sum(p.numel() for p in model.out_head.parameters())
print(f"Número de parâmetros treináveis considerando weight tying: {total_params_gpt2:,}")

Número de parâmetros treináveis considerando weight tying: 123,822,336


In [33]:
total_size_bytes = total_params * 4 # cada parâmetro ocupando 4 bytes
total_size_mb = total_size_bytes / (1024 * 1024)
print(f"Tamanho total do modelo: {total_size_mb:.2f} MB")

Tamanho total do modelo: 619.58 MB


In [34]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    
    # idx é o vetor de índices no contexto atual (batch, n_tokens)
    
    for _ in range(max_new_tokens):
        # cortar contexto atual se exceder o context size
        # se suporta 5 tokens e o context size é 10
        # só os últimos 5 tokens são usados como contexto
        idx_condition = idx[:, -context_size:]
        
        with torch.no_grad():
            logits = model(idx_condition)
        
        # foca apenas na ultima linha de cada batch e as junta
        # (batch, n_tokens, vocab_size) -> (batch, vocab_size)
        logits = logits[:, -1, :]
        
        probs = torch.softmax(logits, dim=-1) # aplica softmax nas colunas
        idx_next = torch.argmax(probs, dim=-1, keepdim=True) # pega index da entrada com maior probabilidade
        
        # juntar index com maior probabilidade ao input
        idx = torch.cat((idx, idx_next), dim=1) # (batch, n_tokens + 1)
        
    return idx
        

In [35]:
start_context = "Hello, I am"
encoded = tokenizer.encode(start_context)
print("encoded:", encoded)
encoded_tensor = torch.tensor(encoded).unsqueeze(0)
print("encoded tensor shape:", encoded_tensor.shape)

encoded: [15496, 11, 314, 716]
encoded tensor shape: torch.Size([1, 4])


In [36]:
model.eval() # bypass nas camadas de dropout, normalization pois não estamos treinando
out = generate_text_simple(
    model=model, 
    idx=encoded_tensor, 
    max_new_tokens=6, 
    context_size=GPT_CONFIG_124M["context_length"]
)
print("out:", out) # tamanho 10 pois tinhamos 4 de entrada e 6 como numero maximo

out: tensor([[15496,    11,   314,   716, 13240, 11381,  4307,  7640, 16620, 34991]])


In [37]:
decoded_text = tokenizer.decode(out.squeeze(0).tolist())
print(decoded_text)

Hello, I am Laur inhab DistrinetalkQueue


In [38]:
def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # adicionar dimensão de batch
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remover a dimensão de batch
    return tokenizer.decode(flat.tolist())

start_context = "Every effort moves you"

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text: ", token_ids_to_text(token_ids, tokenizer))

Output text:  Every effort moves you rentingetic wasnم refres RexMeCHicular stren


In [39]:
# Definindo loss function
inputs = torch.tensor([[16833, 3626, 6100], # "every effort moves"
                       [40, 1107, 588]])    # "I really like"

targets = torch.tensor([[3626, 6100, 345],  # "effort moves you"
                        [1107, 588, 11311]]) # "really like chocolate"

with torch.no_grad():
    logits = model(inputs)

probs = torch.softmax(logits, dim=-1) # probabilidade de cada token no vocabulario
print(probs.shape)

torch.Size([2, 3, 50257])


In [40]:
token_ids = torch.argmax(probs, dim=-1, keepdim=True)
print("Token IDs:\n", token_ids)

Token IDs:
 tensor([[[16657],
         [  339],
         [42826]],

        [[49906],
         [29669],
         [41751]]])


In [41]:
print(f"Targets batch 1: {token_ids_to_text(targets[0], tokenizer)}")
print(f"Outputs batch 1: {token_ids_to_text(token_ids[0].flatten(), tokenizer)}")

Targets batch 1:  effort moves you
Outputs batch 1:  Armed heNetflix


In [42]:
text_idx = 0
# primeiro batch, verificando linhas 0, 1 e 2 pelas probabilidades dos indices dos targets
target_probs_1 = probs[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 1:", target_probs_1)

text_idx = 1
target_probs_2 = probs[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 2:", target_probs_2)

Text 1: tensor([    0.0001,     0.0000,     0.0000])
Text 2: tensor([    0.0000,     0.0001,     0.0000])


In [43]:
log_probs = torch.log(torch.cat((target_probs_1, target_probs_2)))
print("Log probs:\n", log_probs)

Log probs:
 tensor([ -9.5042, -10.3796, -11.3677, -11.4798,  -9.7764, -12.2561])


In [44]:
# probabilidade média para cada token
avg_log_probs = torch.mean(log_probs)
print("Avg log probs:\n", avg_log_probs)

Avg log probs:
 tensor(-10.7940)


In [45]:
neg_avg_log_probs = avg_log_probs * -1
print("Negative log probs:\n", neg_avg_log_probs)

Negative log probs:
 tensor(10.7940)


In [46]:
logits_flat = logits.flatten(0, 1)
targets_flat = targets.flatten()

print("Flattened logits:\n", logits_flat.shape)
print("Flattened targets:\n", targets_flat.shape)

loss = torch.nn.functional.cross_entropy(logits_flat, targets_flat)
print("Loss:", loss)

Flattened logits:
 torch.Size([6, 50257])
Flattened targets:
 torch.Size([6])
Loss: tensor(10.7940)


In [47]:
path = "/home/enid/Downloads/the-verdict.txt"

with open(path, "r", encoding="utf-8") as f:
    text = f.read()

In [48]:
total_tokens = len(tokenizer.encode(text))
print(total_tokens)

5145


In [49]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        
        token_ids = tokenizer.encode(txt, allowed_special={'<|endoftext|>'})
        
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
    
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    
    tokenizer = tiktoken.get_encoding("gpt2")
    
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,    # desconsidera último batch se for muito pequeno
        num_workers=num_workers
    )
    return dataloader

In [50]:
train_ratio = 0.9
split_idx = int(train_ratio * len(text))
train_data = text[:split_idx]
val_data = text[split_idx:]

torch.manual_seed(123)

train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)

val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=False,
)

In [51]:
train_tokens = 0
val_tokens = 0

for input_batch, target_batch in train_loader:
    train_tokens += input_batch.numel()

for input_batch, target_batch in val_loader:
    val_tokens += input_batch.numel()
    
print("Training tokens: ", train_tokens)
print("Validation tokens: ", val_tokens)
print("All tokens: ", train_tokens + val_tokens)

Training tokens:  4608
Validation tokens:  512
All tokens:  5120


In [52]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval();

In [53]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
    return loss

def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # reduzir numero de batches para se encaixar no numero total de batches do data loader
        # se num_batches exceder o numero de batches do data loader
        num_batches = min(num_batches, len(data_loader))
    
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches

In [54]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
torch.manual_seed(123)

with torch.no_grad(): # desabilitar gradient tracking pois não estamos treinando
    train_loss = calc_loss_loader(train_loader, model, device)
    val_loss = calc_loss_loader(val_loader, model, device)
    
print("Training loss: ", train_loss)
print("Validation loss: ", val_loss)

Training loss:  10.98758347829183
Validation loss:  10.981106758117676


In [55]:
def train_model_simple(
        model, 
        train_loader,
        val_loader,
        optimizer,
        device,
        num_epochs,
        eval_freq,
        eval_iter,
        start_context,
        tokenizer
):
    train_losses, val_losses, track_tokens_seen = [], [], []
    tokens_seen, global_step = 0, -1
    
    for epoch in range(num_epochs):
        model.train() # coloca modelo em modo de treinamento
        
        for input_batch, target_batch in train_loader:
            optimizer.zero_grad() # resetar loss gradients da iteração anterior
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward() # calcular gradientes de perda
            optimizer.step() # Atualizar pesos do modelo com gradientes de perda
            tokens_seen += input_batch.numel()
            global_step += 1
            
            # mostra loss a cada 5 batches processados
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter
                )
                
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                track_tokens_seen.append(tokens_seen)
                print(f"Epoch {epoch+1} (Step {global_step:06d}) Train Loss: {train_loss:.3f} Val Loss: {val_loss:.3f}")
        
        # após cada época, printar quais tokens llm está gerando
        generate_and_print_sample(
            model, tokenizer, device, start_context
        )
        
    return train_losses, val_losses, track_tokens_seen

In [56]:
def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss

In [57]:
def generate_and_print_sample(model, tokenizer, device, start_context):
    model.eval()
    context_size = model.pos_emb.weight.shape[0]
    encoded = text_to_token_ids(start_context, tokenizer).to(device)
    
    with torch.no_grad():
        token_ids = generate_text_simple(
            model=model,
            idx=encoded,
            max_new_tokens=50,
            context_size=context_size,
        )
    decoded_text = token_ids_to_text(token_ids, tokenizer)
    print(decoded_text.replace("\n", " "))
    model.train()

In [58]:
import time
start_time = time.time()

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)

num_epochs = 10
train_losses, val_losses, track_tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=5, eval_iter=5,
    start_context="Every effort moves you", tokenizer=tokenizer
)

end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print("Training completed in {execution_time_minutes:.2f} minutes")

Epoch 1 (Step 000000) Train Loss: 9.783 Val Loss: 9.927
Epoch 1 (Step 000005) Train Loss: 8.050 Val Loss: 8.333
Every effort moves you,,,,,,,,,,,,.                                     
Epoch 2 (Step 000010) Train Loss: 6.758 Val Loss: 7.046
Epoch 2 (Step 000015) Train Loss: 6.142 Val Loss: 6.630
Every effort moves you, and, and, and, and, and, and, and.                                   
Epoch 3 (Step 000020) Train Loss: 13.925 Val Loss: 14.195
Epoch 3 (Step 000025) Train Loss: 5.513 Val Loss: 6.420
Every effort moves you, and to the"                                             
Epoch 4 (Step 000030) Train Loss: 5.165 Val Loss: 6.381
Epoch 4 (Step 000035) Train Loss: 4.753 Val Loss: 6.316
Every effort moves you.                           "I"I"I"I it's"I"I"I"I"I"I
Epoch 5 (Step 000040) Train Loss: 4.627 Val Loss: 6.394
Every effort moves you, I had been.                       "II me, and he had the donkey.           
Epoch 6 (Step 000045) Train Loss: 4.026 Val Loss: 6.260
Epoch 6 (Step 

In [59]:
vocab = {
    "closer": 0,
    "every": 1,
    "effort": 2,
    "forward": 3,
    "inches": 4,
    "moves": 5,
    "pizza": 6,
    "toward": 7,
    "you": 8,
}

inverse_vocab = {v: k for k, v in vocab.items()}

next_token_logits = torch.tensor(
    [4.51, 0.89, -1.90, 6.75, 1.63, -1.62, -1.89, 6.28, 1.79]
)

probs = torch.softmax(next_token_logits, dim=0)

In [60]:
# Temperature scaling
# Realiza sample para definir próximo token - nem sempre será o mesmo
def print_sampled_tokens(probs):
    torch.manual_seed(123)
    sample = [torch.multinomial(probs, num_samples=1).item() for i in range(1000)]
    sampled_ids = torch.bincount(torch.tensor(sample))
    for i, freq in enumerate(sampled_ids):
        print(f"{freq} x {inverse_vocab[i]}")
    
print_sampled_tokens(probs)

71 x closer
2 x every
0 x effort
544 x forward
2 x inches
1 x moves
0 x pizza
376 x toward
4 x you


In [61]:
def softmax_with_temperature(logits, temperature):
    scaled_logits = logits / temperature
    return torch.softmax(scaled_logits, dim=0)

temperatures = [1, 0.1, 5] # original, mais confiança e menos confiança

scaled_probs = [softmax_with_temperature(next_token_logits, T) for T in temperatures]

In [62]:
# Top-k sampling
top_k = 3
top_logits, top_pos = torch.topk(next_token_logits, top_k)

In [63]:
new_logits = torch.where(
    condition=next_token_logits < top_logits[-1], # logits com probabilidade menor que a menor probabilidade do terceiro elemento k são desconsiderados
    input=torch.tensor(float("-inf")),
    other=next_token_logits
)
print(new_logits)

tensor([4.5100,   -inf,   -inf, 6.7500,   -inf,   -inf,   -inf, 6.2800,   -inf])


In [64]:
topk_probs = torch.softmax(new_logits, dim=0)
print(topk_probs)

tensor([0.0615, 0.0000, 0.0000, 0.5775, 0.0000, 0.0000, 0.0000, 0.3610, 0.0000])


In [65]:
# idx é a sequência de input
def generate(model, idx, max_new_tokens, context_size, temperature=0.0, top_k=None, eos_id=None): # eos = end of sequence
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:] # : serve para manter as amostras, pega todos os últimos tokens de cada batch
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :] # torna logits (batch, vocabulary_size), pega apenas último token
        # previsão é baseada apenas no último token
        
        if top_k is not None:
            top_logits, _ = torch.topk(logits, top_k)
            min_val = top_logits[:, -1]
            logits = torch.where(logits < min_val, torch.tensor(float("-inf")).to(logits.device), logits)
            
        if temperature > 0.0:
            logits = logits / temperature
            
            probs = torch.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
        else:
            idx_next = torch.argmax(probs, dim=-1, keepdim=True)
        
        if idx_next == eos_id: # parar de gerar se encontrar end of sequence
            break
        
        idx = torch.cat((idx, idx_next), dim=1) # append do novo token à sequência até atingir max_new_tokens
    
    return idx

In [66]:
torch.manual_seed(123)

token_ids = generate(
    model=model,
    idx=text_to_token_ids("Every effort moves you", tokenizer),
    max_new_tokens=15,
    context_size=GPT_CONFIG_124M["context_length"],
    top_k=25,
    temperature=1.4
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves you'd he wouldn down--it, and thought one: "Whenating with


In [67]:
# Salvando estados do modelo e optimizer
torch.save({
    "model_state_dict": model.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    },
    "model_and_optimizer.pth"
)

checkpoint = torch.load("model_and_optimizer.pth")
model = GPTModel(GPT_CONFIG_124M)
model.load_state_dict(checkpoint["model_state_dict"]) # carregando pesos
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
model.train()

/tmp/ipykernel_9046/2734612657.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("model_and_optimizer.pth")


GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(256, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (transformer_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_key): Linear(in_features=768, out_features=768, bias=False)
        (W_value): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_

In [76]:
# Define model configurations in a dictionary for compactness
model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

# Copy the base configuration and update with specific model settings
model_name = "gpt2-small (124M)"  # Example model name
NEW_CONFIG = GPT_CONFIG_124M.copy()
NEW_CONFIG.update(model_configs[model_name])

NEW_CONFIG.update({"context_length": 1024, "qkv_bias": True})
gpt = GPTModel(NEW_CONFIG)
gpt.eval();

In [77]:
from get_weights import *
settings, params = get_gpt2_weights("./models/gpt/")

In [78]:
def assign(left, right):
    if left.shape != right.shape:
        raise ValueError(f"Erro de dimensões. Esq: {left.shape}, Dir: {right.shape}")
    return torch.nn.Parameter(torch.tensor(right))

In [81]:
import numpy as np

def load_weights_into_gpt(gpt, params):
    gpt.pos_emb.weight = assign(gpt.pos_emb.weight, params['wpe'])
    gpt.tok_emb.weight = assign(gpt.tok_emb.weight, params['wte'])
    
    for b in range(len(params["blocks"])):
        q_w, k_w, v_w = np.split(
            (params["blocks"][b]["attn"]["c_attn"])["w"], 3, axis=-1)
        gpt.trf_blocks[b].att.W_query.weight = assign(
            gpt.trf_blocks[b].att.W_query.weight, q_w.T)
        gpt.trf_blocks[b].att.W_key.weight = assign(
            gpt.trf_blocks[b].att.W_key.weight, k_w.T)
        gpt.trf_blocks[b].att.W_value.weight = assign(
            gpt.trf_blocks[b].att.W_value.weight, v_w.T)

        q_b, k_b, v_b = np.split(
            (params["blocks"][b]["attn"]["c_attn"])["b"], 3, axis=-1)
        gpt.trf_blocks[b].att.W_query.bias = assign(
            gpt.trf_blocks[b].att.W_query.bias, q_b)
        gpt.trf_blocks[b].att.W_key.bias = assign(
            gpt.trf_blocks[b].att.W_key.bias, k_b)
        gpt.trf_blocks[b].att.W_value.bias = assign(
            gpt.trf_blocks[b].att.W_value.bias, v_b)

        gpt.trf_blocks[b].att.out_proj.weight = assign(
            gpt.trf_blocks[b].att.out_proj.weight, 
            params["blocks"][b]["attn"]["c_proj"]["w"].T)
        gpt.trf_blocks[b].att.out_proj.bias = assign(
            gpt.trf_blocks[b].att.out_proj.bias, 
            params["blocks"][b]["attn"]["c_proj"]["b"])

        gpt.trf_blocks[b].ff.layers[0].weight = assign(
            gpt.trf_blocks[b].ff.layers[0].weight, 
            params["blocks"][b]["mlp"]["c_fc"]["w"].T)
        gpt.trf_blocks[b].ff.layers[0].bias = assign(
            gpt.trf_blocks[b].ff.layers[0].bias, 
            params["blocks"][b]["mlp"]["c_fc"]["b"])
        gpt.trf_blocks[b].ff.layers[2].weight = assign(
            gpt.trf_blocks[b].ff.layers[2].weight, 
            params["blocks"][b]["mlp"]["c_proj"]["w"].T)
        gpt.trf_blocks[b].ff.layers[2].bias = assign(
            gpt.trf_blocks[b].ff.layers[2].bias, 
            params["blocks"][b]["mlp"]["c_proj"]["b"])

        gpt.trf_blocks[b].norm1.scale = assign(
            gpt.trf_blocks[b].norm1.scale, 
            params["blocks"][b]["ln_1"]["g"])
        gpt.trf_blocks[b].norm1.shift = assign(
            gpt.trf_blocks[b].norm1.shift, 
            params["blocks"][b]["ln_1"]["b"])
        gpt.trf_blocks[b].norm2.scale = assign(
            gpt.trf_blocks[b].norm2.scale, 
            params["blocks"][b]["ln_2"]["g"])
        gpt.trf_blocks[b].norm2.shift = assign(
            gpt.trf_blocks[b].norm2.shift, 
            params["blocks"][b]["ln_2"]["b"])

    gpt.final_norm.scale = assign(gpt.final_norm.scale, params["g"])
    gpt.final_norm.shift = assign(gpt.final_norm.shift, params["b"])
    gpt.out_head.weight = assign(gpt.out_head.weight, params["wte"])



In [82]:
load_weights_into_gpt(gpt, params)
gpt.to(device)

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(1024, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=True)
        (W_key): Linear(in_features=768, out_features=768, bias=True)
        (W_value): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=7

In [83]:
torch.manual_seed(123)

token_ids = generate(
    model=gpt,
    idx=text_to_token_ids("Every effort moves you", tokenizer).to(device),
    max_new_tokens=25,
    context_size=NEW_CONFIG["context_length"],
    top_k=50,
    temperature=1.4
)

print("Output: \n", token_ids_to_text(token_ids, tokenizer))

Output: 
 Every effort moves you toward an equal share for each vote plus half. Inequality is often not an accurate representation of human worth; to know the


In [3]:
import pandas as pd

df = pd.read_csv("./data/SMSSpamCollection", sep="\t", header=None, names=["Label", "Text"])

In [6]:
def create_balanced_dataset(df):
    num_spam = df[df["Label"] == "spam"].shape[0]
    ham_subset = df[df["Label"] == "ham"].sample(num_spam, random_state=123)
    balanced_df = pd.concat([ham_subset, df[df["Label"] == "spam"]])
    return balanced_df

balanced_df = create_balanced_dataset(df)
print(balanced_df["Label"].value_counts())

Label
ham     747
spam    747
Name: count, dtype: int64


In [7]:
balanced_df["Label"] = balanced_df["Label"].map({"ham": 0, "spam": 1})

In [8]:
def random_split(df, train_frac, validation_frac):
    df = df.sample(frac=1, random_state=123).reset_index(drop=True)
    
    train_end = int(len(df) * train_frac)
    validation_end = int(len(df) * validation_frac)
    
    train_df = df[:train_end]
    validation_df = df[train_end:validation_end]
    test_df = df[validation_end:]
    return train_df, validation_df, test_df

train_df, validation_df, test_df = random_split(balanced_df, 0.7, 0.1)

In [9]:
train_df.to_csv("./data/train.csv", index=None)
validation_df.to_csv("./data/validation.csv", index=None)
test_df.to_csv("./data/test.csv", index=None)